```
dx download chr-subset-genes-variants/all_variants_chr21.tsv
```

In [5]:
library(dplyr)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [6]:
vars.df = read.table('all_variants_chr21.tsv', as.is=TRUE, header=TRUE)
head(vars.df)

,chr,start,end,type,variant_id
,<int>,<int>,<int>,<chr>,<chr>
1,21,5051593,5051594,INS,nssv14017801
2,21,5064052,5066138,DEL,nssv14300595
3,21,5064052,5066138,DEL,nssv14301211
4,21,5064052,5066138,DEL,nssv14301212
5,21,5064052,5066138,DEL,nssv14301213
6,21,5064052,5066138,DEL,nssv14432733


# nstd102 inc. ClinGen CNVs
```
dx download Annotation/nstd102\ \(ClinVarSV\ -\ includes\ ClinGen\ CNVs\)/nstd102.GRCh38.variant_call.tsv.gz
```

In [3]:
clin.df = read.table('nstd102.GRCh38.variant_call.tsv.gz', as.is=TRUE, skip=1, comment='', sep='\t', header=TRUE)
head(clin.df)

Warning message in scan(file = file, what = what, sep = sep, quote = quote, dec = dec, :
“EOF within quoted string”
Warning message in scan(file = file, what = what, sep = sep, quote = quote, dec = dec, :
“number of items read is not a multiple of the number of columns”


,X.variant_call_accession,variant_call_id,variant_call_type,experiment_id,sample_id,sampleset_id,assembly,chr,contig,outer_start,⋯,remap_alignment,remap_best_within_cluster,remap_coverage,remap_diff_chr,remap_failure_code,external_links,evidence,sequence,clinical_significance,clinical_source
,<chr>,<chr>,<chr>,<int>,<lgl>,<int>,<chr>,<chr>,<chr>,<int>,⋯,<chr>,<int>,<dbl>,<int>,<chr>,<chr>,<lgl>,<chr>,<chr>,<chr>
1,nssv8639265,RCV000076578_96551,deletion,1,NA,1,GRCh38,2,,NA,⋯,,NA,NA,NA,,"ClinGen:CA331581,ClinVar:RCV000076578.2,PubMed:11598466,PubMed:15604628,PubMed:20301390,PubMed:23408351,PubMed:23535968,PubMed:23788249,PubMed:24310308,PubMed:24493721,PubMed:25003300,PubMed:25070057,PubMed:25356965,PubMed:25452455,PubMed:25645574,PubMed:25711197,PubMed:27854360",NA,,Pathogenic,ClinVar
2,nssv8639267,RCV000203260_215728,duplication,1,NA,1,GRCh38.p12,8,,NA,⋯,First Pass,0,1,NA,,"ClinVar:RCV000203260.1,PubMed:20301641",NA,,Pathogenic,ClinVar
3,nssv8639268,RCV000203462_216747,deletion,1,NA,1,GRCh38.p12,2,,NA,⋯,First Pass,0,1,NA,,"ClinVar:RCV000203462.1,PubMed:20301339",NA,,Pathogenic,ClinVar
4,nssv8639269,RCV000201517_214150,deletion,1,NA,1,GRCh38,2,,NA,⋯,,NA,NA,NA,,ClinVar:RCV000201517.1,NA,,Uncertain significance,ClinVar
5,nssv8639270,RCV000203882_222922,deletion,1,NA,1,GRCh38,7,,NA,⋯,,NA,NA,NA,,ClinVar:RCV000203882.1,NA,,Likely pathogenic,ClinVar
6,nssv8639272,RCV000144263_166057,deletion,1,NA,1,GRCh38,3,,NA,⋯,,NA,NA,NA,,"ClinGen:CA277981,ClinVar:RCV000144263.1,PubMed:20301627,dbSNP:rs1553721650",NA,,Pathogenic,ClinVar


In [10]:
clin.df %>% mutate(variant_id=X.variant_call_accession, sv_clinical_significance=clinical_significance) %>% 
    filter(variant_id %in% vars.df$variant_id) %>% select(variant_id, sv_clinical_significance) %>% 
    write.table('sv.clinical.variants.chr21.nstd120.tsv', row.names=FALSE, sep='\t', quote=FALSE)

```
dx upload sv.clinical.variants.chr21.nstd120.tsv --path chr-subset-genes-variants/
dx upload make-misc-tsvs.ipynb --path chr-subset-genes-variants/
```

# Compact ClinVar TSV

```
dx download chr-subset-genes-variants/final_clinvar_dbvar_results.txt
```

In [13]:
clinvar = read.table('final_clinvar_dbvar_results.txt', as.is=TRUE, sep='\t')
colnames(clinvar) = c('variant_id', 'effect')
head(clinvar)

,variant_id,effect
,<chr>,<chr>
1,nssv15216117,Pathogenic
2,nssv14267889,Pathogenic
3,nssv14267890,Pathogenic
4,nssv14407870,Pathogenic
5,nssv15136031,Likely_benign
6,nssv15136031,Benign


In [15]:
clinvar.sum = clinvar %>% group_by(variant_id, effect) %>% summarize(n=n())
head(clinvar.sum)

`summarise()` regrouping output by 'variant_id' (override with `.groups` argument)



variant_id,effect,n
<chr>,<chr>,<int>
essv100139,Pathogenic,1
essv100355,Pathogenic,1
essv100680,Benign,26
essv100680,Benign/Likely_benign,7
essv100680,Conflicting_interpretations_of_pathogenicity,9
essv100680,Likely_benign,74


In [16]:
write.table(clinvar.sum, file='final_clinvar_dbvar_results_summary.txt', sep='\t', row.names=FALSE, quote=FALSE)